# Quick Start for Network Wrangler

This notebook should give you a high-level overview of using Wrangler to do the following:  

1. Import a network to wrangler
2. Query network features 
3. Change attributes 
4. Write out resulting network  
5. Make change to a network based on a project card 

In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
import geopandas as gpd

from network_wrangler import RoadwayNetwork
from network_wrangler import ProjectCard
from network_wrangler import TransitNetwork

from ipywidgets import HBox, VBox, Output
from IPython.display import display, HTML

%config IPCompleter.greedy=True
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

In [ ]:
#if you need to reset variables in the notebook
%reset

### 1. Importing and viewing a network in the standard format 

1. Locate network folder and files 

In [ ]:
STPAUL_DIR = os.path.join(os.getcwd(),'../','examples','stpaul')

# Roadway Network files
STPAUL_SHAPE_FILE = os.path.join(STPAUL_DIR,"shape.geojson")
STPAUL_LINK_FILE = os.path.join(STPAUL_DIR,"link.json")
STPAUL_NODE_FILE = os.path.join(STPAUL_DIR,"node.geojson")

2. Read in the network

In [ ]:
net = RoadwayNetwork.read(
    link_file= STPAUL_LINK_FILE, 
    node_file=STPAUL_NODE_FILE, 
    shape_file=STPAUL_SHAPE_FILE, 
    fast=True        # fast=True means that the network isn't validated against its schema when it is read in. You will want to try both ways.
)

In [ ]:
transit_net = TransitNetwork.read(feed_path=STPAUL_DIR)

3. Look at the first three rows of each network component

In [ ]:
net.links_df[:3]

In [ ]:
net.nodes_df[:3]

In [ ]:
net.shapes_df[:3]

In [ ]:
transit_net.feed.routes[:3]

In [ ]:
transit_net.feed.stop_times[:3]

In [ ]:
transit_net.feed.frequencies[0:3]

In [ ]:
transit_net.feed.trips[0:3]

In [ ]:
transit_net.feed.stops[:3]

In [ ]:
stops_gdf = gpd.GeoDataFrame(
    transit_net.feed.stops, geometry=gpd.points_from_xy(transit_net.feed.stops.stop_lon, transit_net.feed.stops.stop_lat))
stops_gdf.plot(column='wheelchair_boarding')

In [ ]:
transit_net.feed.shapes[:5]
# this can be a geodataframe, but have it turned off for now b/c of issues with partridge

## 2. Query Network Features

Most of Network wrangler operates by querying a portion of the network and then making changes to it.  

Queries take the form of Python Dictionaries or can be read-in as YAML which is then converted to a python dictionary.

If a query has already been made, it is stored in the network so that it will not take up computing power to find it again.

### Highway Segment Selection

Highway selections have three required components: 'link', 'A', and 'B'.  

'link' must either have a specified name, or an 'osmid'

'A' and 'B' must specify some sort of unique identifier which is found in the data structure (AKA, it should return a single node).

If not all the links connecting A to B have the 'name' or an 'osmid', wrangler will connect the dots from A-->B using as many streets with the correct name/osmid as possible.

In [ ]:
easy_highway_selection = \
{
    'link':[
        {'name': ['6th', 'Sixth', 'sixth']}, #find streets that have one of the various forms of 6th
        ],
     'A':{'osm_node_id': '187899923'}, # start searching for segments at A
     'B':{'osm_node_id': '187865924'}, # end at B
}

In [ ]:
# querying with the selection will return the links that are selected
net.select_roadway_features(easy_highway_selection)

In [ ]:
# wrangler has other information about the facility selection stored under a unique key, including a graph and a route
A_id, B_id = net.orig_dest_nodes_foreign_key(easy_highway_selection)
easy_key = (ProjectCard.build_link_selection_query(easy_highway_selection, unique_model_link_identifiers = ['model_link_id']), A_id, B_id)

G               = net.selections[easy_key]['graph']
selection_route = net.selections[easy_key]['route']
selected_links  = net.selections[easy_key]['selected_links']

#### Visualizing the selection to verify it is correct

In [ ]:
try:
    import osmnx as ox
except:
    !conda install osmnx
try:
    import folium
except:
    !conda install folium

## todo make this part of wrangler
## todo visualize with shapes rather than links
    
def folium_node(node, node_name, color='white', icon = ''):
        node_marker = folium.Marker(location=[node['y'],node['x']], 
                                    icon=folium.Icon(icon = icon, color=color), 
                                    tooltip=node_name,
                                   )
        return node_marker
    
def map_selection(stored_selection, A_name, B_name):
        '''
        Shows which links are selected
        '''
        
        
        m = ox.plot_graph_folium(stored_selection['graph'], 
                                 edge_color='blue',
                                 edge_width=5, 
                                 edge_opacity = 0.6,
                                 tiles='cartodbdark_matter')
        
        A     = stored_selection['graph'].nodes[A_name]
        B     = stored_selection['graph'].nodes[B_name]
        
        folium_node(A, A_name, color="green", icon = 'play').add_to(m)
        folium_node(B, B_name, color="pink", icon = 'star').add_to(m)
        
        for _, row in stored_selection['selected_links'].iterrows():
            pl = ox.plot.make_folium_polyline(edge=row, edge_color="green", edge_width=7,
                                      edge_opacity=0.8)
            pl.add_to(m)
            
        
        return m

In [ ]:
map_selection(net.selections[easy_key], A_id, B_id)

# this map shows the links that were considered because of their OSMid or their name in blue, 
# and the final selection in green

### More complex selections

You can also select facilities based on their properties.  This selection is tiered from the name/osmid selection.

In [ ]:
multi_criteria_selection = \
{
    'link':[
        {'name': ['6th', 'Sixth', 'sixth']}, #find streets that have one of the various forms of 6th
        {'lanes': [1,2]}, # only select links that are either 1 or 2 lanes
         ],
     'A':{'osm_node_id': '187899923'}, # start searching for segments at A
     'B':{'osm_node_id': '187865924'}, # end at B
}
net.select_roadway_features(multi_criteria_selection )

In [ ]:
A_id, B_id = net.orig_dest_nodes_foreign_key(multi_criteria_selection)
multi_criteria_key = (ProjectCard.build_link_selection_query(multi_criteria_selection,unique_model_link_identifiers= ['model_link_id']), A_id, B_id)
map_selection(net.selections[multi_criteria_key], A_id, B_id)

### Transit Selection


Transit trip selection can currently happen by querying trips, routes, etc.

In [ ]:
simple_transit_selection = {"route_id": "365-111"}
selected_trips = transit_net.select_transit_features(simple_transit_selection)

transit_net.feed.trips[transit_net.feed.trips.trip_id.isin(selected_trips)]

In [ ]:
simple_transit_selection = {"route_short_name": "3"}
selected_trips = transit_net.select_transit_features(simple_transit_selection)

transit_net.feed.routes[transit_net.feed.trips.trip_id.isin(selected_trips)]

#### Visualize selection on map

Note that this is a mouthful of code.  I have a todo to write a wrapper around this so that you can easily visualize with an api of something like:  `transit_net.select_transit_features(simple_transit_selection).plot()`

In [ ]:
#NOTE not currently working because of partridge/geopandas issues. Working to resolve.
all_routes = transit_net.feed.shapes.plot(color='gray')
transit_net.feed.shapes[transit_net.feed.trips.trip_id.isin(selected_trips)].plot(ax=all_routes, color="red")

## 3. Change Feature Attributes

### Highway Feature Change

In [ ]:
selected_highway = {
    "link": [{"name": ["I 35E"]}],
    "A": {"osm_node_id": "961117623"},  
    "B": {"osm_node_id": "2564047368"}
}
selected_links   = net.select_roadway_features(selected_highway)

A_id, B_id = net.orig_dest_nodes_foreign_key(selected_highway)
selected_highway_key = (ProjectCard.build_link_selection_query(selected_highway,unique_model_link_identifiers=['model_link_id']), A_id, B_id)
map_selection(net.selections[selected_highway_key ], A_id, B_id)

In [ ]:
net.links_df.loc[selected_links]['lanes']

In [ ]:
change = {
    'properties': [
      {
          'property': 'lanes',
          'set': 2,
      }
    ]
}

In [ ]:
net.apply_roadway_feature_change(selected_links,change['properties'])
net.links_df.loc[selected_links]['lanes']

#### You can also increment from the existing value using the field `change`
And optionally, you can call out what you think the existing value should be so that it fails if it isn't the case.

In [ ]:
change = {
    'properties': [
      {
          'property': 'lanes',
          'existing': 2,
          'change': 1,
      }
    ]
}

net.apply_roadway_feature_change(selected_links,change['properties'])
net.links_df.loc[selected_links]['lanes']

### Transit Feature Change


In [ ]:
transit_selection = {"route_short_name": "3"}
change = {
    'properties': [
      {
          'property': 'headway_secs',
          'set': 600, # ten minute headways
      }
    ]
}
# show existing headways
transit_net.feed.frequencies[
    transit_net.feed.trips.trip_id.isin(
        transit_net.select_transit_features(transit_selection)
    )
]

In [ ]:
transit_net.apply_transit_feature_change(
            transit_net.select_transit_features(transit_selection), change['properties']
        )

# show revised headways
transit_net.feed.frequencies[
    transit_net.feed.trips.trip_id.isin(
        transit_net.select_transit_features(simple_transit_selection)
    )
]

## 4 - Write out Networks

In [ ]:
OUTPATH = "/Users/Elizabeth/Downloads"

In [ ]:
net.write(filename="my_net", path=OUTPATH)

In [ ]:
transit_net.write(path = OUTPATH)

## 5 - Do the whole process with project cards

#### Roadway

In [ ]:
roadway_project_card_file = os.path.join(
        STPAUL_DIR, "project_cards", '3_multiple_roadway_attribute_change.yml'
    )
    
roadway_project_card = ProjectCard.read(roadway_project_card_file)

net.apply_roadway_feature_change(
    net.select_roadway_features(roadway_project_card.facility),
    roadway_project_card.properties
)


#### Transit

In [ ]:
transit_project_card_file = os.path.join(
    STPAUL_DIR, "project_cards", '8_simple_transit_attribute_change.yml'
)

transit_project_card = ProjectCard.read(transit_project_card_file)
transit_net.apply_transit_feature_change(
    transit_net.select_transit_features(transit_project_card.facility), 
    transit_project_card.properties
)

# Scenarios

Can create a scenario from a base scenario which specifies any base networks if applicable.
You can add projects and apply them all.
You have access to the projects that have been applied.

In [ ]:
from network_wrangler import Scenario
scen = Scenario.create_scenario(
    base_scenario={"road_net":net, "transit_net":transit_net}, 
    project_cards_list= [roadway_project_card]
    )

scen.apply_all_projects()

In [ ]:
scen.applied_projects